# Reactant demo

In [1]:
using Tenet
using EinExprs
using Reactant
using Enzyme
using BenchmarkTools
using Adapt

using LinearAlgebra
BLAS.set_num_threads(1)

using Random
Random.seed!(0)

TaskLocalRNG()

In [2]:
tn = rand(TensorNetwork, 15, 3; dim=(16,16))

path = einexpr(tn; optimizer=Exhaustive())

@benchmark contract(tn; path)

BenchmarkTools.Trial: 438 samples with 1 evaluation.
 Range (min … max):  10.154 ms … 202.250 ms  ┊ GC (min … max): 0.00% … 94.66%
 Time  (median):     10.385 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.387 ms ±   9.307 ms  ┊ GC (mean ± σ):  5.02% ±  5.30%

  █▆▅▃ ▄▄▂                                                      
  ████████▇▅▁▄▁▁▁▁▁▄▄▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▄▁▁▁▁▁▅▄▆▇█▇ ▆
  10.2 ms       Histogram: log(frequency) by time      17.7 ms <

 Memory estimate: 9.41 MiB, allocs estimate: 3303.

In [3]:
tn′ = adapt(Reactant.ConcreteRArray, tn)

g = Reactant.compile(Tuple(tensors(tn′))) do ts...
    _tn = TensorNetwork(ts)
    contract(_tn; path)
end

@benchmark g(tensors(tn′)...)

BenchmarkTools.Trial: 590 samples with 1 evaluation.
 Range (min … max):  7.593 ms … 17.671 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.174 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.481 ms ±  1.115 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▃▇█▇▆▆▆▄▃                                                 
  ▄▇██████████▆▄▁▄▄▁▁▄▁▄▁▁▅▄▄▄▄▁▁▄▁▅▅▇▅▄▄▁▅▄▄▁▁▁▁▇▁▅▄▅▄▄▁▁▁▄ ▇
  7.59 ms      Histogram: log(frequency) by time     13.5 ms <

 Memory estimate: 496 bytes, allocs estimate: 7.

In [4]:
function f(x...)
    _tn = TensorNetwork(x)
    contract(_tn; path)
end

∇g = Reactant.compile(Tuple(tensors(tn′))) do x...
    dx = Enzyme.make_zero.(x)
    Enzyme.autodiff(Reverse, f, Active, Duplicated.(x,dx)...)
    return dx
end

@benchmark ∇g(tensors(tn′)...)

BenchmarkTools.Trial: 181 samples with 1 evaluation.
 Range (min … max):  25.377 ms …  33.153 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     27.563 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.610 ms ± 782.976 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                     ▁ ▁▁ ▂▄▆▄▂▂▃█ ▄▃▁ ▁▁                       
  ▃▁▁▁▁▃▁▁▁▁▁▃▁▁▁▃▇▄▆█████████████▇██████▇▄▄▆▄▇▃▃▄▁▃▁▃▃▁▁▁▁▁▁▃ ▃
  25.4 ms         Histogram: frequency by time         29.7 ms <

 Memory estimate: 2.64 KiB, allocs estimate: 63.

In [5]:
re∇g = Reactant.compile(Tuple(tensors(tn′))) do x...
    dx = Enzyme.make_zero.(x)
    primal = Enzyme.autodiff(ReverseWithPrimal, f, Active, Duplicated.(x,dx)...)
    return (primal, dx)
end

@benchmark re∇g(tensors(tn′)...)

BenchmarkTools.Trial: 197 samples with 1 evaluation.
 Range (min … max):  23.327 ms … 44.784 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     24.845 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.364 ms ±  2.029 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

        ▄▆█▆▆▅▂▁▁      ▁                                       
  ▅▁▁▁▁▆█████████▆▁▁▆▅▆█▆▅▁▁▁▅▅▁▅▁▁▁▁▁▁▁▅▁▁▁▅▁▁▅▁▅▁▁▁▁▁▁▁▁▁▁▆ ▅
  23.3 ms      Histogram: log(frequency) by time      32.8 ms <

 Memory estimate: 2.77 KiB, allocs estimate: 67.